In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
# get a glimpse on the data
digits = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
# as we can see here, no columns should be modified or deleted
digits.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# as we can see, values go from 0 to 254
digits.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [5]:
# we separate the target ('label') from the data
y = digits.label
X = digits.drop('label', axis=1)

In [6]:
# normalizing the values before splitting the data
X = X / 255

In [7]:
# splitting the data into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [9]:
X_train.shape

(33600, 784)

In [17]:
# transform the data directly into tensors
import torch

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_valid_tensor = torch.tensor(X_valid.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.long)

In [18]:
# build the Multilayer Perceptron
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_dl = DataLoader(train_ds,8, shuffle=True)

model = nn.Sequential(
    nn.Linear(X_train.shape[1],284), # I used 284 as arbitrary
    nn.ReLU(),
    nn.Linear(284, 102), # again, arbitrary numbers
    nn.ReLU(),
    nn.Linear(102,10)
)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

num_epochs = 5
for epoch in range(num_epochs):
    for x, y in train_dl:
        predictions = model(x).squeeze()
        loss = loss_fn(predictions, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
        predictions = model(X_train_tensor).squeeze()
        epoch_loss = loss_fn(predictions, y_train_tensor)
        print(f"Loss: {epoch_loss}")

loss = loss_fn(model(X_valid_tensor), y_valid_tensor)
print(f"Validation Loss: {loss}")

Loss: 0.3196874260902405
Loss: 0.22865420579910278
Loss: 0.1738647073507309
Loss: 0.1286851018667221
Loss: 0.10548284649848938
Validation Loss: 0.14175330102443695


In [20]:
# prepare the test data
digits_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

X_test = digits_test / 255
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

In [24]:
# get the predictions
model.eval()

with torch.no_grad():
    final_predictions = model(X_test_tensor).squeeze()
    final_predictions = torch.argmax(final_predictions, dim=1)

In [25]:
submission = pd.DataFrame({
    'ImageId': range(1,digits_test.shape[0]+1),
    'Label': final_predictions.numpy()
})

submission.to_csv('submission.csv', index=False)